In [ ]:
datatrain_path = "/home/dev2/Sanjana/stride/train.csv"
glove_path = "/home/dev2/Embeddings/Glove/glove.840B.300d.txt"
datatest_path="/home/dev2/Sanjana/stride/test.csv"
# data_apth = './data.xlsx'

In [2]:
import pandas as pd
import numpy as np

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /home/dev2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:

df = pd.read_csv(datatrain_path)
df
# cleaning data
df.paragraphs= df.paragraphs.str.lower()
df.paragraphs= df.paragraphs.str.replace(r'[^a-zA-Z\s]+|X{2,}', '')
df.paragraphs= df.paragraphs.str.replace(r'_x005F_x000E_', '')

# df = df.head(660)
# df = df.loc[640:650] 



In [4]:
withstop_x = df['paragraphs'].values
withoutstop_x=[]
y = df['Tags'].values
for i in withstop_x:
    text_tokens = word_tokenize(i)
    text1 = [word for word in text_tokens if not word in stopwords.words()]
    filtered = (" ").join(text1)
    withoutstop_x.append(filtered)
    
print(len(withstop_x))
print(len(y))
    

3657
3657


In [5]:
x=withoutstop_x
len(x), len(y)


(3657, 3657)

In [6]:
tokenizer = keras.preprocessing.text.Tokenizer(oov_token='unk', lower=True)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)
word_index = tokenizer.word_index
word_index['pad'] = 0

In [7]:
word_index

{'unk': 1,
 'please': 2,
 'get': 3,
 'would': 4,
 'like': 5,
 'email': 6,
 'call': 7,
 'discuss': 8,
 'us': 9,
 'know': 10,
 'make': 11,
 'link': 12,
 'need': 13,
 'time': 14,
 'week': 15,
 'send': 16,
 'let': 17,
 'meeting': 18,
 'give': 19,
 'could': 20,
 'forward': 21,
 'next': 22,
 'dont': 23,
 'tomorrow': 24,
 'help': 25,
 'day': 26,
 'new': 27,
 'see': 28,
 'go': 29,
 'may': 30,
 'available': 31,
 'think': 32,
 'address': 33,
 'information': 34,
 'back': 35,
 'meet': 36,
 'today': 37,
 'monday': 38,
 'free': 39,
 'going': 40,
 'questions': 41,
 'getting': 42,
 'receive': 43,
 'lets': 44,
 'sure': 45,
 'together': 46,
 'well': 47,
 'use': 48,
 'try': 49,
 'th': 50,
 'check': 51,
 'work': 52,
 'issues': 53,
 'visit': 54,
 'look': 55,
 'contact': 56,
 'possible': 57,
 'talk': 58,
 'join': 59,
 'click': 60,
 'list': 61,
 'right': 62,
 'enron': 63,
 'add': 64,
 'find': 65,
 'last': 66,
 'review': 67,
 'friday': 68,
 'youd': 69,
 'pm': 70,
 'schedule': 71,
 'gas': 72,
 'might': 73,
 'm

In [8]:
sent_max_len = 300
word_max_len = 10

In [9]:
padded_sequences = keras.preprocessing.sequence.pad_sequences(maxlen=sent_max_len, sequences=sequences, value=word_index["pad"], padding='post', truncating='post')

In [10]:
chars = set([w for word in word_index.keys() for w in word])
n_chars = len(chars)
print(n_chars)

26


In [11]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["unk"] = 1
char2idx["pad"] = 0

In [12]:
sentence_list = [sent.split() for sent in x]
from tqdm import tqdm
char_sequences = []
for sentence in tqdm(sentence_list):
    sent_seq = []
    for i in range(sent_max_len):
        word_seq = []
        for j in range(word_max_len):
            try:
                word_seq.append(char2idx.get(sentence[i][j]))
            except:
                word_seq.append(char2idx.get("pad"))
        sent_seq.append(word_seq)
    char_sequences.append(np.array(sent_seq))

100%|██████████| 3657/3657 [00:06<00:00, 603.68it/s]


In [13]:
print(y)
encoder = LabelEncoder()
y = encoder.fit_transform(y)

['No' 'No' 'No' ... 'Yes' 'Yes' 'Yes']


In [14]:
X_train_word, X_test_word, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.1, random_state=85)
X_train_char, X_test_char, _, _ = train_test_split(char_sequences, y, test_size=0.1, random_state=85)

In [15]:
X_train_word=padded_sequences
y_train=y
X_train_char=char_sequences
_=y

In [16]:
embeddings_matrix = np.zeros((len(word_index)+1, 300))
with open(glove_path) as f:
    for line in f:
        values = line.split(' ')
        word = str(values[0].strip())
        vector = np.asarray(values[1:], dtype='float32')
        if vector is not None:
            if word in word_index.keys():
                i = word_index.get(word)
                embeddings_matrix[i] = vector

In [17]:
# building model

# input and embedding for words
word_input = keras.models.Input(shape=(sent_max_len,))
embeddings_word = keras.layers.Embedding(input_dim=len(word_index)+1, output_dim=300,
                            input_length=sent_max_len, weights=[embeddings_matrix],
                                         trainable=False, mask_zero=True)(word_input)

# input and embeddings for characters
char_input = keras.models.Input(shape=(sent_max_len, word_max_len,))
embeddings_char = keras.layers.TimeDistributed(keras.layers.Embedding(input_dim=len(char2idx), 
                        output_dim=10, input_length=word_max_len, mask_zero=True))(char_input)

# character LSTM to get word encodings by characters
char_encodings = keras.layers.TimeDistributed(keras.layers.LSTM(units=64, return_sequences=False,
                                recurrent_dropout=0.3))(embeddings_char)

x = keras.layers.concatenate([embeddings_word, char_encodings])
x = keras.layers.SpatialDropout1D(0.3)(x)

main_lstm = keras.layers.Bidirectional(keras.layers.LSTM(units=128, return_sequences=False,
                               recurrent_dropout=0.6))(x)
out = keras.layers.Dense(len(encoder.classes_), activation="softmax")(main_lstm)

model = keras.models.Model([word_input, char_input], out)

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["acc"])

In [19]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 300, 10)      0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 300, 10, 10)  280         input_2[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 300, 300)     1897800     input_1[0][0]                    
____________________________________________________________________________________________

In [20]:
history = model.fit([X_train_word,
                    np.array(X_train_char).reshape((len(X_train_char), sent_max_len, word_max_len))],
                    y_train, batch_size=32, epochs=50, validation_split=0.2) 

Instructions for updating:
Use tf.cast instead.
Train on 2925 samples, validate on 732 samples
Epoch 1/50
2925/2925 [==============================] - 87s 30ms/step - loss: 0.5232 - acc: 0.7569 - val_loss: 0.8114 - val_acc: 0.4713
Epoch 2/50
2925/2925 [==============================] - 86s 29ms/step - loss: 0.4438 - acc: 0.8106 - val_loss: 0.9744 - val_acc: 0.3661
Epoch 3/50
2925/2925 [==============================] - 85s 29ms/step - loss: 0.3985 - acc: 0.8250 - val_loss: 1.0066 - val_acc: 0.4167
Epoch 4/50
2925/2925 [==============================] - 86s 29ms/step - loss: 0.3822 - acc: 0.8332 - val_loss: 1.2431 - val_acc: 0.3060
Epoch 5/50
2925/2925 [==============================] - 89s 30ms/step - loss: 0.3570 - acc: 0.8451 - val_loss: 0.8003 - val_acc: 0.5451
Epoch 6/50
2925/2925 [==============================] - 86s 29ms/step - loss: 0.3292 - acc: 0.8585 - val_loss: 1.0284 - val_acc: 0.4617
Epoch 7/50
2925/2925 [==============================] - 86s 29ms/step - loss: 0.3066 - ac

In [47]:
# prediction
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

X_test_word, X_test_char, y_test
y_pred = model.predict([X_test_word,
                        np.array(X_test_char).reshape((len(X_test_char), sent_max_len, word_max_len))])
from sklearn.preprocessing import LabelBinarizer
encoder2 = LabelBinarizer()
yy = encoder2.fit_transform(y_test)
y_classes = y_pred.argmax(axis=-1)
encoder3 = LabelBinarizer()
yyy = encoder3.fit_transform(y_classes)
print(classification_report(yy, yyy))
print(confusion_matrix(y_test, y_classes))
text="I want to hear what M. Cash said, and who else you are meeting with."
text = text.replace(r'[^a-zA-Z\s]+|X{2,}', '')
text1 = tokenizer.texts_to_sequences([text])
text2 = keras.preprocessing.sequence.pad_sequences(maxlen=sent_max_len, sequences=text1, value=word_index["pad"], padding='post', truncating='post')
text2.shape
sent_seq = []
text3 = text.split()
for i in range(sent_max_len):
    word_seq = []
    for j in range(word_max_len):
        try:
            if char2idx.get(text3[i][j]) is None:
                raise Exception        
            word_seq.append(char2idx.get(text3[i][j]))
        except:
            word_seq.append(char2idx.get("pad"))
    sent_seq.append(word_seq)
char_text2 = np.array(sent_seq)
prediction = model.predict([text2,
                        char_text2.reshape(1,300,10)])
print(prediction)
print("hi")
classes=['no','yes']
print(classes[int(prediction.argmax(axis=1))])

prediction[0][int(prediction.argmax(axis=1))]

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       197
           1       1.00      0.80      0.89       169

    accuracy                           0.91       366
   macro avg       0.93      0.90      0.90       366
weighted avg       0.92      0.91      0.91       366

[[197   0]
 [ 34 135]]
[[0.99041384 0.0095862 ]]
hi
no


0.99041384

In [43]:
model.save("binary_model_new.h5")
import pickle
with open('word_tokenizer_binary_model.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
# df.to_excel('binary_model-trained-data.xlsx', index=False)

In [44]:
pwd()

'/home/dev2/Sanjana/stride'

In [45]:
default_model_path = '/home/dev2/Sanjana/stride'

In [46]:
from keras.models import load_model

In [36]:
import pickle
with open('character_tokenizer_binary.pickle','wb') as handle:
    pickle.dump(char2idx, handle, protocol=pickle.HIGHEST_PROTOCOL)